# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of skills that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

We encourage you to implement your own versions of the planner that fit your user needs.

Read more about it [here](https://aka.ms/sk/concepts/planner)

In [11]:
# !python -m pip install semantic-kernel==0.2.7.dev0

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI backend used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

### Setting Up the Planner
Here we will be using a BasicPlanner that will create a sequential plan to be evaluated in order. The ouput of a function becomes the inputs of the next function. 

In [2]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [3]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

In [4]:
text_skill

{'lowercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7bf3d0>,
 'trim': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7bfb50>,
 'trim_end': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7bfc70>,
 'trim_start': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7bfd90>,
 'uppercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7bfeb0>}

In [5]:
summarize_skill

{'MakeAbstractReadable': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda79afd0>,
 'Notegen': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b11c0>,
 'Summarize': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b1550>,
 'Topics': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b1880>}

In [6]:
writer_skill

{'Acronym': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b1bb0>,
 'AcronymGenerator': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b1ee0>,
 'AcronymReverse': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b6250>,
 'Brainstorm': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b65e0>,
 'EmailGen': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b6970>,
 'EmailTo': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b6e20>,
 'EnglishImprover': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b80d0>,
 'NovelChapter': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b8940>,
 'NovelChapterWithNotes': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7b8dc0>,
 'NovelOutline': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7bb520>,
 'Rewrite': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1fbda7bb670>,
 'ShortPoem':

Define your ASK. What do you want the Kernel to do?

In [15]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""
original_plan = await planner.create_plan_async(ask, kernel)

In [16]:
print(original_plan.generated_plan)

    {
        "input": "Valentine's Day Date Ideas",
        "subtasks": [
            {"function": "WriterSkill.Brainstorm"},
            {"function": "WriterSkill.Translate", "args": {"language": "French"}},
            {"function": "TextSkill.uppercase"}
        ]
    }


You can see that the Planner took my ask and converted it into an JSON-based plan detailing
how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call 
in order to fulfill the user ask. The output of each step of the plan becomes the `input` to the next function. 

Let's also define an inline skill and have it be available to the Planner.
Be sure to give it a function name and skill name.

In [17]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

Let's update our ask using this new skill.

In [18]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

In [19]:
print(new_plan.generated_plan)

    {
        "input": "Valentine's Day Date Ideas",
        "subtasks": [
            {"function": "WriterSkill.Brainstorm"},
            {"function": "ShakespeareSkill.shakespeare"},
            {"function": "WriterSkill.Translate", "args": {"language": "French"}},
            {"function": "TextSkill.uppercase"}
        ]
    }


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [20]:
results = await planner.execute_plan_async(new_plan, kernel)

In [21]:
print(results)

1. ENSEMBLE DANS UNE CLASSE DE CUISINE,
2. NOUS AVONS UN PIQUE-NIQUE ROMANTIQUE,
3. UNE JOURNÉE DE SPA DE PLAISIR CHOYÉ SOIT,
4. ÉCOUTONS UN CONCERT EN PLEIN AIR,
5. NOUS DEVONS NOUS PROMENER DANS UN MUSÉE OU UNE GALERIE D'ART,
6. UN VOYAGE EN VOITURE EST UN MUST,
7. UN DÎNER ROMANTIQUE SOIT NOTRE DÉLICE DU SOIR,
8. ALLONS À UNE CAVE À VIN EN VOL,
9. NOUS DEVONS ALLER ENSEMBLE SUR DES PATINS À GLACE,
10. CONTEMPLONS LE COUCHER DE SOLEIL ENSEMBLE.
